In [14]:
import os
import pandas as pd
import numpy as np
import json

In [28]:
# Load the JSON file
with open(r"C:\2.Documents\3.LSU\BreakThrough AI Tech\2.Fall AI Studio\Project - Personal work\data\abcd_v1.10.json", "r") as f: #Raw string
    data = json.load(f)

display(data.keys())

dict_keys(['train', 'dev', 'test'])

In [64]:
#Convert train portion of datafile into a dataframe
train_df = pd.DataFrame(data["train"]).drop(columns="delexed")#Removed the delexed row, its not needed
train_df.columns

Index(['convo_id', 'scenario', 'original'], dtype='object')

In [66]:
#Created a transcript column(series), to make the text easier for the llm to process
train_df["transcript"] = train_df["original"].apply(
    lambda turns: " ".join([f"{speaker}: {text}" for speaker, text in turns])
)
train_df["transcript"]


0       agent: Hi! agent: How can I help you? customer...
1       agent: good afternoon, how can I help you? cus...
2       customer: HEY HO! agent: good afternoon, how c...
3       agent: Welcome to AcmeBrands! How can I help y...
4       agent: Hello, how can i help you customer: Hel...
                              ...                        
8029    agent: Welcome to AcmeBrands. How may I help? ...
8030    agent: Hello, how can I help you today? custom...
8031    agent: Hello, how may I help you? customer: Hi...
8032    customer: Hi agent: Hello, what can I do for y...
8033    agent: how may i help you customer: Hello, Eve...
Name: transcript, Length: 8034, dtype: object

In [68]:
#See the different tuples in scenario
display(train_df["scenario"].keys(), train_df["scenario"][0].keys())

RangeIndex(start=0, stop=8034, step=1)

dict_keys(['personal', 'order', 'product', 'flow', 'subflow'])

In [130]:
#Display the different categories within each tuple
train_df["scenario"][0]

{'personal': {'customer_name': 'crystal minh',
  'email': 'cminh730@email.com',
  'member_level': 'bronze',
  'phone': '(977) 625-2661',
  'username': 'cminh730'},
 'order': {'street_address': '6821 1st ave',
  'full_address': '6821 1st ave  san mateo, ny 75227',
  'city': 'san mateo',
  'num_products': '1',
  'order_id': '3348917502',
  'packaging': 'yes',
  'payment_method': 'credit card',
  'products': "[{'brand': 'michael_kors', 'product_type': 'jeans', 'amount': 94, 'image_url': 'images/michael_kors-jeans.jpeg'}]",
  'purchase_date': '2019-11-06',
  'state': 'ny',
  'zip_code': '75227'},
 'product': {'names': ['michael_kors jeans'], 'amounts': [94]},
 'flow': 'product_defect',
 'subflow': 'return_size'}

In [ ]:
#Create a traditional dataframe with all the categories and the transcription per conversation
rows = []
for _, row in train_df.iterrows():
    convo_id = row["convo_id"]
    scenario = row["scenario"] #Contains 3 dicts
    transcript = row["transcript"]

    #Flatten scenario keys
    personal = scenario.get("personal", {})
    order = scenario.get("order", {})
    product = scenario.get("product", {})
    
    rows.append({
        "convo_id": row["convo_id"],
        "customer_name": personal.get("customer_name"),
        "email": personal.get("email"),
        "member_level": personal.get("member_level"),
        "phone": personal.get("phone"),
        "username": personal.get("username"),
        "street_address": order.get("street_address"),
        "full_address": order.get("full_address"),
        "city": order.get("city"),
        "num_products": order.get("num_products"),
        "order_id": order.get("order_id"),
        "packaging": order.get("packaging"),
        "payment_method": order.get("payment_method"),
        "products": order.get("products"),
        "purchase_date": order.get("purchase_date"),
        "state": order.get("state"),
        "zip_code": order.get("zip_code"),
        "names": product.get("names"),
        "amounts": product.get("amounts"),
        "flow": scenario.get("flow"),
        "subflow": scenario.get("subflow"),
        "transcript": row["transcript"],
    })

structured_tain_df = pd.DataFrame(rows)
structured_tain_df.head()

,convo_id,customer_name,email,member_level,phone,username,street_address,full_address,city,num_products,...,payment_method,products,purchase_date,state,zip_code,names,amounts,flow,subflow,transcript
0,3592,crystal minh,cminh730@email.com,bronze,(977) 625-2661,cminh730,6821 1st ave,"6821 1st ave san mateo, ny 75227",san mateo,1,...,credit card,"[{'brand': 'michael_kors', 'product_type': 'je...",2019-11-06,ny,75227,[michael_kors jeans],[94],product_defect,return_size,agent: Hi! agent: How can I help you? customer...
1,9489,alessandro phoenix,aphoenix939@email.com,gold,(727) 760-7806,aphoenix939,8865 lexington ave,"8865 lexington ave la fayette, tx 86229",la fayette,1,...,credit card,"[{'brand': 'michael_kors', 'product_type': 'sh...",2019-11-20,tx,86229,[michael_kors shirt],[69],product_defect,refund_status,"agent: good afternoon, how can I help you? cus..."
2,3695,joyce wu,None,bronze,(859) 787-9085,None,9998 lincoln ave,"9998 lincoln ave raleigh, mi 14573",raleigh,None,...,None,None,None,mi,14573,[],[],storewide_query,timing_4,"customer: HEY HO! agent: good afternoon, how c..."
3,5798,crystal minh,cminh948@email.com,silver,(656) 332-6844,cminh948,9723 primrose ave,"9723 primrose ave newark, mi 85971",newark,3,...,None,"[{'brand': 'tommy_hilfiger', 'product_type': '...",2020-03-30,mi,85971,"[tommy_hilfiger shirt, michael_kors shirt, mic...","[94, 79, 54]",shipping_issue,manage,agent: Welcome to AcmeBrands! How can I help y...
4,3647,norman bouchard,None,guest,(480) 983-3323,None,7092 1st ave,"7092 1st ave raleigh, ca 46681",raleigh,3,...,None,"[{'brand': 'michael_kors', 'product_type': 'je...",None,ca,46681,"[michael_kors jeans, tommy_hilfiger jeans, tom...","[49, 79, 54]",storewide_query,pricing_3,"agent: Hello, how can i help you customer: Hel..."


#### Okay, I have a transcription of every conversation, and each category. What i need to do is build a system, where i feed the transcripts into an api, and spit out the correct answers to the categories(the ones in scenario dicts)[step 1: pipeline]. I will need to test that they match the real answers[step 2 evaluation: accuracy will be a focus in the project], and find out how to have them placed in a dataframe for viewing like I did up top[step 3: presentation].

In [148]:
structured_tain_df.columns.tolist()

['convo_id',
 'customer_name',
 'email',
 'member_level',
 'phone',
 'username',
 'street_address',
 'full_address',
 'city',
 'num_products',
 'order_id',
 'packaging',
 'payment_method',
 'products',
 'purchase_date',
 'state',
 'zip_code',
 'names',
 'amounts',
 'flow',
 'subflow',
 'transcript']